
# Associate Ca2+ signal with spindles for each session & subsessions using crossregistration

Load packages

In [1]:
cd "C:/Users/Manip2/SCRIPTS/Code python audrey/code python aurelie/interfaceJupyter/minian"

C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian


In [2]:
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy.interpolate import interp2d
import pickle
import os
from scipy.interpolate import griddata
import logging

%matplotlib widget

from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer
from IPython.display import display
from ipyfilechooser import FileChooser


from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

Load sleep score and Ca2+ time series numpy arrays

In [3]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in strore")
    #dpath = "/Users/mb/Documents/Syntuitio/AudreyHay/PlanB/ExampleRedLines/2022_08_06/13_30_01/My_V4_Miniscope/"
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\AnalysedMarch2023\Gaelle\Baseline_recordi…

Stored 'dpath' (str)
Stored 'dpath' (str)
Stored 'dpath' (str)


In [15]:
folder_base = Path(dpath)

nb_sessions = sum(1 for p in folder_base.iterdir() if p.is_dir() and p.name.startswith("session"))

try:
    mfile = open(folder_base / f'mappingsAB.pkl', 'rb')
    mapping = pickle.load(mfile)
    print('mappingsAB.pkl opened')
except:
    mfile = open(folder_base / f'mappings.pkl', 'rb')
    mapping = pickle.load(mfile)
    print('mappings.pkl opened')

sessions = []
subsessions = []
nb_minian_total=0
dict_Calcium = {}
dict_Spike = {}
dict_SWRprop = {}
dict_Spindleprop_PFC = {}
dict_Spindleprop_S1 = {}
dict_Stamps = {}
dict_StampsMiniscope = {}
dict_TodropFile = {}

for y in range(1, nb_sessions+1):
    session= 'session' + str(y)
    print(session)
    sessions.append(session)
    folder_mini = folder_base / f'session{y}/V4_Miniscope'
    nb_subsessions = sum(1 for p in folder_mini.iterdir() if p.is_dir() and p.name.startswith("session"))
    SWRproperties = folder_base / f'session{y}/OpenEphys/SWRproperties_AB.csv'
    Spindleproperties_PFC = folder_base / f'session{y}/OpenEphys/Spindlesproperties_PFC_AB.csv'
    Spindleproperties_S1 = folder_base / f'session{y}/OpenEphys/Spindlesproperties_S1_AB.csv'
    StampsFile = folder_base / f'session{y}/SynchroFile.xlsx'
    StampsMiniscopeFile = folder_mini / f'timeStamps.csv'
    if nb_subsessions!=0:
        for x in range(1, nb_subsessions+1):            
            subsession= "session"  + str(y) + str(x)
            subsessions.append(subsession)    
            minian_ds = open_minian(folder_mini / subsession / f'minian')      # OR minianAB
            dict_Calcium[subsession] = minian_ds['C'] # calcium traces 
            dict_Spike[subsession] = minian_ds['S'] # estimated spikes
            dict_SWRprop[subsession]  = pd.read_csv(SWRproperties)
            dict_Spindleprop_PFC[subsession]  = pd.read_csv(Spindleproperties_PFC)
            dict_Spindleprop_S1[subsession]  = pd.read_csv(Spindleproperties_S1)
            dict_Stamps[subsession]  = pd.read_excel(StampsFile)
            dict_StampsMiniscope[subsession]  = pd.read_csv(StampsMiniscopeFile)
            try:
                TodropFile = folder_mini / subsession / f'minian/TodropFileAB.json'
                with open(TodropFile, 'r') as f:
                    unit_to_drop = json.load(f)
                    dict_TodropFile[subsession]  = unit_to_drop
            except:
                TodropFile = folder_mini / subsession / f'minian/TodropFile.json'
                with open(TodropFile, 'r') as f:
                    unit_to_drop = json.load(f)
                    dict_TodropFile[subsession]  = unit_to_drop
            nb_minian_total+=1
            print(nb_minian_total)
    else:
        minian_ds = open_minian(folder_mini / f'minian')            # OR minianAB
        dict_Calcium[session] = minian_ds['C'] # calcium traces 
        dict_Spike[session] = minian_ds['S'] # estimated spikes
        dict_SWRprop[session]  = pd.read_csv(SWRproperties)
        dict_Spindleprop_PFC[session]  = pd.read_csv(Spindleproperties_PFC)
        dict_Spindleprop_S1[session]  = pd.read_csv(Spindleproperties_S1)
        dict_Stamps[session]  = pd.read_excel(StampsFile)
        dict_StampsMiniscope[session]  = pd.read_csv(StampsMiniscopeFile)
        try:
            TodropFile = folder_mini / f'minian/TodropFileAB.json'
            with open(TodropFile, 'r') as f:
                unit_to_drop = json.load(f)
                dict_TodropFile[session]  = unit_to_drop
        except:
            TodropFile = folder_mini / f'minian/TodropFile.json'
            with open(TodropFile, 'r') as f:
                unit_to_drop = json.load(f)
                dict_TodropFile[session]  = unit_to_drop
        nb_minian_total+=1  
        print(nb_minian_total)

mappingsAB.pkl opened
session1


C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  arr = list(xr.open_zarr(arr_path).values())[0]
C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, c

1
session2


C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  arr = list(xr.open_zarr(arr_path).values())[0]
C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, c

2
session3


...

3
session4


C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  arr = list(xr.open_zarr(arr_path).values())[0]
C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, c

4
session5


C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  arr = list(xr.open_zarr(arr_path).values())[0]
C:\Users\Manip2\SCRIPTS\Code python audrey\code python aurelie\interfaceJupyter\minian\minian\utilities.py:342: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, c

5


Cross registration results

In [16]:
mapping

session                                     variable                 group
   session1 session2 session3 session4 session5 distance                 group
0     152.0     13.0      NaN      NaN      NaN      NaN  (session1, session2)
1     155.0     14.0      NaN      NaN      NaN      NaN  (session1, session2)
2       NaN      NaN      NaN      5.0    401.0      NaN  (session4, session5)
3       NaN      NaN      NaN      6.0    423.0      NaN  (session4, session5)
4       NaN      NaN      NaN      7.0    433.0      NaN  (session4, session5)
5       NaN      NaN      NaN      2.0    180.0      NaN  (session4, session5)
6     292.0      NaN      NaN      NaN      NaN      NaN           (session1,)
7     394.0      NaN      NaN      NaN      NaN      NaN           (session1,)
8     523.0      NaN      NaN      NaN      NaN      NaN           (session1,)
9     139.0      NaN      NaN      NaN      NaN      NaN           (session1,)
10    115.0      NaN      NaN      NaN      NaN      NaN           (session1,)
11    572.0      NaN      NaN      NaN      NaN      NaN           (session1,)
12      NaN      0.0      NaN      NaN      NaN      NaN           (session2,)
13      NaN      2.0      NaN      NaN      NaN      NaN           (session2,)
14      NaN      4.0      NaN      NaN      NaN      NaN           (session2,)
15      NaN      7.0      NaN      NaN      NaN      NaN           (session2,)
16      NaN      8.0      NaN      NaN      NaN      NaN           (session2,)
17      NaN      9.0      NaN      NaN      NaN      NaN           (session2,)
18      NaN     10.0      NaN      NaN      NaN      NaN           (session2,)
19      NaN     12.0      NaN      NaN      NaN      NaN           (session2,)
20      NaN     15.0      NaN      NaN      NaN      NaN           (session2,)
21      NaN     16.0      NaN      NaN      NaN      NaN           (session2,)
22      NaN      NaN    851.0      NaN      NaN      NaN           (session3,)
23      NaN      NaN    874.0      NaN      NaN      NaN           (session3,)
24      NaN      NaN    939.0      NaN      NaN      NaN           (session3,)
25      NaN      NaN    717.0      NaN      NaN      NaN           (session3,)
26      NaN      NaN    655.0      NaN      NaN      NaN           (session3,)
27      NaN      NaN      NaN      8.0      NaN      NaN           (session4,)
28      NaN      NaN      NaN      4.0      NaN      NaN           (session4,)
29      NaN      NaN      NaN      NaN    515.0      NaN           (session5,)
30      NaN      NaN      NaN      NaN    196.0      NaN           (session5,)
31      NaN      NaN      NaN      NaN    264.0      NaN           (session5,)
32      NaN      NaN      NaN      NaN    233.0      NaN           (session5,)
33      NaN      NaN      NaN      NaN    395.0      NaN           (session5,)
34      NaN      NaN      NaN      NaN    438.0      NaN           (session5,)
35      NaN      NaN      NaN      NaN    155.0      NaN           (session5,)
36      NaN      NaN      NaN      NaN    189.0      NaN           (session5,)

In [17]:
B = mapping['session']
if os.path.basename(folder_base) == 'Purple':
    index = B.columns
    B.columns = index.str.replace('part', 'session2')
        
for c in range(len(B)):
    print('unit n°', c)
    for sess in list(dict_Stamps.keys()):
        print('= unit', int(B[sess][c]), 'in', sess) if math.isnan (float(B[sess][c])) == False else None

unit n° 0
= unit 152 in session1
= unit 13 in session2
unit n° 1
= unit 155 in session1
= unit 14 in session2
unit n° 2
= unit 5 in session4
= unit 401 in session5
unit n° 3
= unit 6 in session4
= unit 423 in session5
unit n° 4
= unit 7 in session4
= unit 433 in session5
unit n° 5
= unit 2 in session4
= unit 180 in session5
unit n° 6
= unit 292 in session1
unit n° 7
= unit 394 in session1
unit n° 8
= unit 523 in session1
unit n° 9
= unit 139 in session1
unit n° 10
= unit 115 in session1
unit n° 11
= unit 572 in session1
unit n° 12
= unit 0 in session2
unit n° 13
= unit 2 in session2
unit n° 14
= unit 4 in session2
unit n° 15
= unit 7 in session2
unit n° 16
= unit 8 in session2
unit n° 17
= unit 9 in session2
unit n° 18
= unit 10 in session2
unit n° 19
= unit 12 in session2
unit n° 20
= unit 15 in session2
unit n° 21
= unit 16 in session2
unit n° 22
= unit 851 in session3
unit n° 23
= unit 874 in session3
unit n° 24
= unit 939 in session3
unit n° 25
= unit 717 in session3
unit n° 26
= u

In [18]:
from bisect import bisect_left

def take_closest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.
    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return 0
    if pos == len(myList):
        return len(myList)
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        return after
    else:
        return before

def take_closest2(myList, myNumber):
    value2 = 10000000
    for ind in range(len(myList)):
        value = abs(myList[ind]-myNumber)
        if value < value2:
            value2 = value
            index = myList[ind]
    return index


def take_closest3(myList, myNumber):
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
        dummy = myList.index(after)
        return dummy
    else:
        dummy = myList.index(before)
        return dummy
    

def is_between(myList, starttime, endtime):
    IsTrue='False'
    for ind in range(len(myList)):
        if starttime <= myList[ind] <= endtime:
            IsTrue='True'
    return IsTrue

Distribute Ca2+ intensity & spikes to vigilance state for each sessions/subsessions

In [20]:
CortexList= ['PFC', 'S1']  #CortexList= ['PFC', 'S1', 'PFC&S1']

for Cortex in CortexList:
    data = {}
    before = 1000 # Max distance in ms between a SWR and a spindle to be considered as Precoupled
    after = 1000 # Max distance in ms between a spindle and a SWR to be considered as Postcoupled
    duration = 0.5 # number of sec before and after the Spdl onset taken into acount
    counter=0
    counter2=0

    norm_freq=10 # final miniscope frequency used for all recordings

    Spindles_GlobalResults= pd.DataFrame(data, columns=['Mice', 'Session','Session_Time','Unique_Unit','UnitNumber','UnitValue','SpdlStatut','SpdlNumber','SpdlDuration (ms)','SWR inside Spdl','CalciumActivityPreference', 'CalciumActivityBefore','CalciumActivityAfter','AUC_calciumBefore','AUC_calciumAfter','SpikeActivityPreference','SpikeActivityBefore','SpikeActivityAfter','AUC_spikeBefore', 'AUC_spikeAfter'])
    SWR_GlobalResults= pd.DataFrame(data, columns=['Mice', 'Session','Session_Time','Unique_Unit','UnitNumber','UnitValue','SWRStatut','SWRNumber','SWRDuration (ms)','SWR inside Spdl','CalciumActivityPreference', 'CalciumActivityBefore','CalciumActivityAfter','AUC_calciumBefore','AUC_calciumAfter','SpikeActivityPreference','SpikeActivityBefore','SpikeActivityAfter','AUC_spikeBefore', 'AUC_spikeAfter'])

    dict_All_ActivityCa_spin={}
    dict_All_ActivityCa_spin_Precoupled={}
    dict_All_ActivityCa_spin_Postcoupled={}
    dict_All_ActivityCa_spin_Uncoupled={}

    dict_All_ActivityCa_swr={}
    dict_All_ActivityCa_swr_Precoupled={}
    dict_All_ActivityCa_swr_Postcoupled={}
    dict_All_ActivityCa_swr_Uncoupled={}

    previousEndTime=0
    InitialStartTime=0

    for i in list(dict_Stamps.keys()):
        cPreCoupled=0
        cPostCoupled=0
        cUnCoupled=0

        cPreCoupledSWR=0
        cPostCoupledSWR=0
        cUnCoupledSWR=0

        # Start time & freq miniscope
        StartTime = list(dict_Stamps[i][0])[0] # in seconds
        minian_freq=list(dict_Stamps[i][0])[2] # in Hz

        # start time session 2
        def Convert(string):
            li = list(string.split(", "))
            li2 = len(li)
            return li2
        stri = dict_Stamps[i][0][3]
        numbdropfr = Convert(stri)

        from ast import literal_eval
        list_droppedframes = literal_eval(dict_Stamps[i][0][3])

        First_frame = 0 #StartTime*minian_freq 
        C=dict_Calcium[i]
        Cupd = C.loc[:, :] #C.loc[:, First_frame:]
        rec_dur = Cupd.shape[1]
        S=dict_Spike[i] 
        Supd = C.loc[:, :] #S.loc[:, :]

        if InitialStartTime==0:
            InitialStartTime=StartTime    
        else:
            if StartTime == InitialStartTime:
                StartTime = previousEndTime + 1/minian_freq #  +1 frame in seconds 
            else:  
                InitialStartTime=StartTime   

        if len(list_droppedframes) > 0:
            numbdropfr = sum(1 for item in list_droppedframes if item < (int(StartTime*minian_freq) + rec_dur) and item > int(StartTime*minian_freq))
        else:
            numbdropfr = 0   

        EndTime = StartTime + ((rec_dur + numbdropfr)/minian_freq) # in seconds
        previousEndTime=EndTime     

        print(i, ': starts at', round(StartTime,1), 's & ends at', round(EndTime,1), 's (', round((rec_dur + numbdropfr)/minian_freq,1), 's duration, ', numbdropfr, 'dropped frames)...') 

        dictnameSpdl=f'dict_Spindleprop_{Cortex}' #change dict according to the cortex 
        dictSpdl = globals()[dictnameSpdl]

        SpipropO=dictSpdl[i]
        SpipropM=SpipropO.copy()
        SWRpropO=dict_SWRprop[i]
        SWRpropM=SWRpropO.copy()
        SpipropM[["peak time", "start time", "end time"]] = SpipropM[["peak time", "start time", "end time"]]-(StartTime*1000)
        SWRpropM[["peak time", "start time", "end time"]] = SWRpropM[["peak time", "start time", "end time"]]-(StartTime*1000)

        AA = C['unit_id']
        copyAA = list(AA.copy())
        unit_to_drop=dict_TodropFile[i]    
        for u in unit_to_drop:
            copyAA.remove(u)
        unit_to_keep = copyAA
        Cupd = Cupd.loc[unit_to_keep,:]
        Supd = Supd.loc[unit_to_keep,:]
        nb_unit = Cupd.shape[0]

        Cseries = Cupd.to_series()
        C_upd_unit_id = Cupd['unit_id'].values
        Sseries = Supd.to_series()

        sentence1= f"... kept values = {C_upd_unit_id}"
        print(sentence1)

        time = range(int(duration*2*minian_freq))
        Half = int(duration*minian_freq)
        TimeStamps_miniscope=list(dict_StampsMiniscope[i]["Time Stamp (ms)"]) # + (StartTime*1000))

        SpipropTrunc = SpipropM[SpipropM["start time"]>0]
        SpipropTrunc = SpipropTrunc[SpipropTrunc["start time"]< (EndTime-StartTime)*1000]
        SWRpropTrunc = SWRpropM[SWRpropM["start time"]>0]
        SWRpropTrunc = SWRpropTrunc[SWRpropTrunc["start time"] < (EndTime-StartTime)*1000]
    
        units = range(nb_unit)
        nb_spindle = SpipropTrunc.shape[0]
        nb_swr = SWRpropTrunc.shape[0]
        for ii, unit in enumerate(units): # for each kept units (cause Cseries/Sseries only have kept units)
            lCseries = np.array(Cseries)[(unit)*(rec_dur + numbdropfr):(unit+1)*(rec_dur + numbdropfr)]
            lSseries = np.array(Sseries)[(unit)*(rec_dur + numbdropfr):(unit+1)*(rec_dur + numbdropfr)]
            ActivityCa_Spin = [] #For each unit  
            ActivityCa_Spin_Precoupled= [] #For each unit 
            ActivityCa_Spin_Postcoupled= [] #For each unit 
            ActivityCa_Spin_Uncoupled= [] #For each unit 
            startSpiList = list(pd.Series(SpipropTrunc["start time"]))
            endSpiList = list(pd.Series(SpipropTrunc["end time"]))

            for Pspin in range(nb_spindle): 
                
                # Get the calcium and spike trace associated with the spdl
                startSpi=startSpiList[Pspin]
                endSpi=endSpiList[Pspin]
                Frame_Spindle_start = take_closest2(TimeStamps_miniscope, startSpi)
                index = TimeStamps_miniscope.index(Frame_Spindle_start)
                CaTrace = list(lCseries[index-Half:index+Half])
                SpTrace = list(lSseries[index-Half:index+Half])   

                if len(CaTrace)<len(time): 
                    print(" /!\ Spindle too close to the begining/end of the recording,", i, ", Spdl n°", Pspin, ", Start Spdl =", int(startSpi), "ms, Start Rec =", int(Frame_Spindle_start), 'ms') if ii==0 else None            
                else:
                    ActivityCa_Spin.append(CaTrace)

                    # Define if that spindle is coupled with a SWR or not
                    Spdl_statut=[]
                    startSWRList = list(pd.Series(SWRpropTrunc["start time"]))
                    if len(startSWRList)>0:
                        startClosest_SWR = take_closest2(startSWRList, startSpi)
                        distance = startClosest_SWR - startSpi
                        if (distance > (- before)) and (distance <  0):
                            Spdl_statut = ['PreCoupled']
                            cPreCoupled+=1 if ii==1 else 0
                            ActivityCa_Spin_Precoupled.append(CaTrace)
                        elif (distance > (0)) and (distance <  after):
                            Spdl_statut = ['PostCoupled']
                            cPostCoupled+=1 if ii==1 else 0
                            ActivityCa_Spin_Postcoupled.append(CaTrace)
                        else:
                            Spdl_statut= ['UnCoupled']
                            cUnCoupled+=1 if ii==1 else 0
                            ActivityCa_Spin_Uncoupled.append(CaTrace)
                    else:
                        Spdl_statut= ['UnCoupled']
                        cUnCoupled+=1 if ii==1 else 0
                        ActivityCa_Spin_Uncoupled.append(CaTrace)

                    Spindles_GlobalResults.loc[counter, 'Mice'] = os.path.basename(folder_base)
                    Spindles_GlobalResults.loc[counter, 'Session'] = i 
                    Spindles_GlobalResults.loc[counter, 'Session_Time'] = None 
                    mapping['session'].columns.tolist()

                    indexMapp = np.where(B[i] == C_upd_unit_id[unit])[0]
                    Spindles_GlobalResults.loc[counter, 'Unique_Unit'] = indexMapp 
                    Spindles_GlobalResults.loc[counter, 'UnitNumber'] = unit 
                    Spindles_GlobalResults.loc[counter, 'UnitValue'] = C_upd_unit_id[unit] 
                    Spindles_GlobalResults.loc[counter, 'SpdlStatut'] = Spdl_statut
                    Spindles_GlobalResults.loc[counter, 'SpdlNumber'] = Pspin
                    Spindles_GlobalResults.loc[counter, 'SpdlDuration (ms)'] = endSpi- startSpi
                    IsTrue=is_between(startSWRList,startSpi, endSpi)
                    Spindles_GlobalResults.loc[counter, 'SWR inside Spdl'] = IsTrue
                    
                    if np.mean(CaTrace[:Half],0) > np.mean(CaTrace[Half:],0):
                        pref='Before'
                    elif np.mean(CaTrace[:Half],0) < np.mean(CaTrace[Half:],0):
                        pref='After' 
                    else:
                        pref='None'
                    Spindles_GlobalResults.loc[counter, 'CalciumActivityPreference'] = pref
                    Spindles_GlobalResults.loc[counter, 'CalciumActivityBefore'] = np.mean(CaTrace[:Half],0)
                    Spindles_GlobalResults.loc[counter, 'CalciumActivityAfter'] = np.mean(CaTrace[Half:],0)
                    Spindles_GlobalResults.loc[counter, 'AUC_calciumBefore'] = np.trapz(CaTrace[:Half],np.arange(0,len(CaTrace[:Half]),1))
                    Spindles_GlobalResults.loc[counter, 'AUC_calciumAfter'] = np.trapz(CaTrace[Half:],np.arange(0,len(CaTrace[Half:]),1))          

                    if np.mean(SpTrace[:Half],0) > np.mean(SpTrace[Half:],0):
                        pref='Before'
                    elif np.mean(SpTrace[:Half],0) < np.mean(SpTrace[Half:],0):
                        pref='After' 
                    else:
                        pref='None'
                    Spindles_GlobalResults.loc[counter, 'SpikeActivityPreference'] = pref
                    Spindles_GlobalResults.loc[counter, 'SpikeActivityBefore'] = np.mean(SpTrace[:Half],0)
                    Spindles_GlobalResults.loc[counter, 'SpikeActivityAfter'] = np.mean(SpTrace[Half:],0)
                    Spindles_GlobalResults.loc[counter, 'AUC_spikeBefore'] = np.trapz(SpTrace[:Half],np.arange(0,len(SpTrace[:Half]),1))
                    Spindles_GlobalResults.loc[counter, 'AUC_spikeAfter'] = np.trapz(SpTrace[Half:],np.arange(0,len(SpTrace[Half:]),1))
                    counter+=1  

            list_ActivityCa= ['ActivityCa_Spin', 'ActivityCa_Spin_Precoupled', 'ActivityCa_Spin_Postcoupled', 'ActivityCa_Spin_Uncoupled']
            list_dict_All_ActivityCa= ['dict_All_ActivityCa_spin', 'dict_All_ActivityCa_spin_Precoupled', 'dict_All_ActivityCa_spin_Postcoupled', 'dict_All_ActivityCa_spin_Uncoupled']
            for it, ActivityCaNames in enumerate(list_ActivityCa): # for each Spdl types
                if len(indexMapp) > 0: #not empty --> cause some units are not in the cross registration..! Need to know why 

                    # All traces for each spindles per Unique unit (according to cross-registration)

                    ActivityCa = locals()[ActivityCaNames]
                    dict_All_ActivityCa = locals()[list_dict_All_ActivityCa[it]]       
                    if len(ActivityCa)>0 :    
                        if np.shape(np.array(ActivityCa))[1] == norm_freq:  #normalize traces to the same frequency rate         
                            ActivityCa= np.reshape(np.array(ActivityCa), (-1, len(np.array(ActivityCa)))) if np.ndim(ActivityCa) == 1 else np.array(ActivityCa)    
                            dict_All_ActivityCa[str(indexMapp)] = np.append(dict_All_ActivityCa[str(indexMapp)], np.array(ActivityCa), axis=0) if str(indexMapp) in dict_All_ActivityCa else np.array(ActivityCa)
                        else:
                            dataO = np.array(ActivityCa)
                            data= np.repeat(dataO, 2, axis=0) if dataO.shape[0] == 1 else dataO
                            x_mesh, y_mesh = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))
                            x_new_mesh, y_new_mesh = np.meshgrid(np.linspace(0, data.shape[1] - 1, norm_freq), np.linspace(0, data.shape[0] - 1, np.shape(data)[0]))
                            resampled_dataO = griddata((x_mesh.flatten(), y_mesh.flatten()), data.flatten(), (x_new_mesh, y_new_mesh), method='linear')
                            resampled_data= resampled_dataO[0,:] if dataO.shape[0] == 1 else resampled_dataO
                            resampled_data= np.reshape(resampled_data, (-1, len(resampled_data))) if np.ndim(resampled_data) == 1 else resampled_data
                            dict_All_ActivityCa[str(indexMapp)] = np.append(dict_All_ActivityCa[str(indexMapp)], np.array(resampled_data), axis=0) if str(indexMapp) in dict_All_ActivityCa else np.array(resampled_data)
                    sentence1bis=""
                else: 
                    sentence1bis=f"/!\ Cell idx {unit} not in the cross registration" if it==1 else ""
                    print(sentence1bis) if it==1 else None
            
            ActivityCa_swr = [] #For each unit  
            ActivityCa_swr_Precoupled= [] #For each unit 
            ActivityCa_swr_Postcoupled= [] #For each unit 
            ActivityCa_swr_Uncoupled= [] #For each unit 

            startSwrList = list(pd.Series(SWRpropTrunc["start time"]))
            endSwrList = list(pd.Series(SWRpropTrunc["end time"]))
            for Pswr in range(nb_swr): 

                # Get the calcium and spike trace associated with the swr

                startSwr=startSwrList[Pswr]
                endSwr=endSwrList[Pswr]
                Frame_SWR_start = take_closest2(TimeStamps_miniscope, startSwr)
                index = TimeStamps_miniscope.index(Frame_SWR_start)
                CaTrace = list(lCseries[index-Half:index+Half])
                SpTrace = list(lSseries[index-Half:index+Half])     
                if len(CaTrace)<len(time): 
                    print("/!\ SWR too close to the begining/end of the recording,", i, ", SWR n°", Pswr, ", Start SWR =",  int(startSwr), "ms, Start Rec =", int(Frame_SWR_start), 'ms') if ii==0 else None 
                else:
                    ActivityCa_swr.append(CaTrace) 

                    # Define if that SWR is coupled with a spdl or not

                    SWR_statut=[]
                    startSpiList = list(pd.Series(SpipropTrunc["start time"]))
                    endSpiList = list(pd.Series(SpipropTrunc["end time"]))
                    if len(startSpiList)>0:
                        startClosest_Spi = take_closest2(startSpiList, startSwr)# + StartTimeIndexSpi])
                        indexSpi = startSpiList.index(startClosest_Spi)
                        endClosest_Spi=endSpiList[indexSpi]
                        distance = startClosest_Spi - startSwr #  + StartTimeIndexSpi]  
                        IsTrue = 'False'             
                        if (distance > (- before)) and (distance <  0):
                            SWR_statut = ['Postcoupled']
                            cPostCoupledSWR+=1 if ii==1 else 0
                            ActivityCa_swr_Postcoupled.append(CaTrace)
                            if startSwr<endClosest_Spi:
                                IsTrue = 'True' #SWR inside the Spindle
                        elif (distance > (0)) and (distance <  after):
                            SWR_statut = ['Precoupled']
                            cPreCoupledSWR+=1 if ii==1 else 0
                            ActivityCa_swr_Precoupled.append(CaTrace)
                        else:
                            SWR_statut= ['UnCoupled']
                            cUnCoupledSWR+=1 if ii==1 else 0
                            ActivityCa_swr_Uncoupled.append(CaTrace)
                    else: 
                        SWR_statut= ['UnCoupled']
                        cUnCoupledSWR+=1 if ii==1 else 0
                        ActivityCa_swr_Uncoupled.append(CaTrace)

                    SWR_GlobalResults.loc[counter2, 'Mice'] = os.path.basename(folder_base)
                    SWR_GlobalResults.loc[counter2, 'Session'] = i 
                    SWR_GlobalResults.loc[counter2, 'Session_Time'] = None 
                    indexMapp = np.where(B[i] == C_upd_unit_id[unit])[0]
                    SWR_GlobalResults.loc[counter2, 'Unique_Unit'] = indexMapp 
                    SWR_GlobalResults.loc[counter2, 'UnitNumber'] = unit 
                    SWR_GlobalResults.loc[counter2, 'UnitValue'] = C_upd_unit_id[unit] 
                    SWR_GlobalResults.loc[counter2, 'SWRStatut'] = SWR_statut
                    SWR_GlobalResults.loc[counter2, 'SWRNumber'] = Pswr
                    SWR_GlobalResults.loc[counter2, 'SWRDuration (ms)'] = endSwr- startSwr
                    SWR_GlobalResults.loc[counter2, 'SWR inside Spdl'] = IsTrue
                    
                    if np.mean(CaTrace[:Half],0) > np.mean(CaTrace[Half:],0):
                        pref='Before'
                    elif np.mean(CaTrace[:Half],0) < np.mean(CaTrace[Half:],0):
                        pref='After' 
                    else:
                        pref='None'
                    SWR_GlobalResults.loc[counter2, 'CalciumActivityPreference'] = pref
                    SWR_GlobalResults.loc[counter2, 'CalciumActivityBefore'] = np.mean(CaTrace[:Half],0)
                    SWR_GlobalResults.loc[counter2, 'CalciumActivityAfter'] = np.mean(CaTrace[Half:],0)
                    SWR_GlobalResults.loc[counter2, 'AUC_calciumBefore'] = np.trapz(CaTrace[:Half],np.arange(0,len(CaTrace[:Half]),1))
                    SWR_GlobalResults.loc[counter2, 'AUC_calciumAfter'] = np.trapz(CaTrace[Half:],np.arange(0,len(CaTrace[Half:]),1))          

                    if np.mean(SpTrace[:Half],0) > np.mean(SpTrace[Half:],0):
                        pref='Before'
                    elif np.mean(SpTrace[:Half],0) < np.mean(SpTrace[Half:],0):
                        pref='After' 
                    else:
                        pref='None'
                    SWR_GlobalResults.loc[counter2, 'SpikeActivityPreference'] = pref
                    SWR_GlobalResults.loc[counter2, 'SpikeActivityBefore'] = np.mean(SpTrace[:Half],0)
                    SWR_GlobalResults.loc[counter2, 'SpikeActivityAfter'] = np.mean(SpTrace[Half:],0)
                    SWR_GlobalResults.loc[counter2, 'AUC_spikeBefore'] = np.trapz(SpTrace[:Half],np.arange(0,len(SpTrace[:Half]),1))
                    SWR_GlobalResults.loc[counter2, 'AUC_spikeAfter'] = np.trapz(SpTrace[Half:],np.arange(0,len(SpTrace[Half:]),1))
                    counter2+=1  

            list_ActivityCa= ['ActivityCa_swr', 'ActivityCa_swr_Precoupled', 'ActivityCa_swr_Postcoupled', 'ActivityCa_swr_Uncoupled']
            list_dict_All_ActivityCa= ['dict_All_ActivityCa_swr', 'dict_All_ActivityCa_swr_Precoupled', 'dict_All_ActivityCa_swr_Postcoupled', 'dict_All_ActivityCa_swr_Uncoupled']
            for it, ActivityCaNames in enumerate(list_ActivityCa): 
                if len(indexMapp) > 0: #not empty --> cause some units are not in the cross registration..! Need to know why 

                    # All traces for each spindles per Unique unit (according to cross-registration)

                    ActivityCa = locals()[ActivityCaNames]
                    dict_All_ActivityCa = locals()[list_dict_All_ActivityCa[it]]                
                    if len(ActivityCa)>0 :  
                        if np.shape(np.array(ActivityCa))[1] == norm_freq:   #normalize traces to the same frequency rate    
                            ActivityCa= np.reshape(np.array(ActivityCa), (-1, len(np.array(ActivityCa)))) if np.ndim(ActivityCa) == 1 else np.array(ActivityCa)    
                            dict_All_ActivityCa[str(indexMapp)] = np.append(dict_All_ActivityCa[str(indexMapp)], np.array(ActivityCa), axis=0) if str(indexMapp) in dict_All_ActivityCa else np.array(ActivityCa)
                        else:
                            dataO = np.array(ActivityCa)
                            data= np.repeat(dataO, 2, axis=0) if dataO.shape[0] == 1 else dataO
                            x_mesh, y_mesh = np.meshgrid(np.arange(data.shape[1]), np.arange(data.shape[0]))
                            x_new_mesh, y_new_mesh = np.meshgrid(np.linspace(0, data.shape[1] - 1, norm_freq), np.linspace(0, data.shape[0] - 1, np.shape(data)[0]))
                            resampled_dataO = griddata((x_mesh.flatten(), y_mesh.flatten()), data.flatten(), (x_new_mesh, y_new_mesh), method='linear')
                            resampled_data= resampled_dataO[0,:] if dataO.shape[0] == 1 else resampled_dataO
                            resampled_data= np.reshape(resampled_data, (-1, len(resampled_data))) if np.ndim(resampled_data) == 1 else resampled_data
                            dict_All_ActivityCa[str(indexMapp)] = np.append(dict_All_ActivityCa[str(indexMapp)], np.array(resampled_data), axis=0) if str(indexMapp) in dict_All_ActivityCa else np.array(resampled_data)
        
        sentence2=f"... in {Cortex}: {nb_spindle} spindles ({cPreCoupled} Pre, {cPostCoupled} Post & {cUnCoupled} Uncoupled Spdl) and {nb_swr} SWR detected ({cPreCoupledSWR} Pre, {cPostCoupledSWR} Post & {cUnCoupledSWR} Uncoupled SWR)"
        print(sentence2)
    sentence3=f"Nb of unique units for {os.path.basename(folder_base)} = {len(dict_All_ActivityCa_spin)}"
    print(sentence3)

session1 : starts at 7.7 s & ends at 2697.3 s ( 2689.6 s duration,  1 dropped frames)...
... kept values = [115 152]
... 0 spindles (0 Pre, 0 Post & 0 Uncoupled Spdl) and 108 SWR detected (0 Pre, 0 Post & 108 Uncoupled SWR)
session2 : starts at 9.2 s & ends at 1071.4 s ( 1062.2 s duration,  0 dropped frames)...
... kept values = [ 4  7  9 10 12 13 14 15 16]
... 0 spindles (0 Pre, 0 Post & 0 Uncoupled Spdl) and 0 SWR detected (0 Pre, 0 Post & 0 Uncoupled SWR)
session3 : starts at 4.1 s & ends at 895.4 s ( 891.3 s duration,  0 dropped frames)...
... kept values = [655 717 851 874 939]
... 0 spindles (0 Pre, 0 Post & 0 Uncoupled Spdl) and 0 SWR detected (0 Pre, 0 Post & 0 Uncoupled SWR)
session4 : starts at 5.2 s & ends at 898.0 s ( 892.8 s duration,  1 dropped frames)...
... kept values = [2 4 5 6 7 8]
... 0 spindles (0 Pre, 0 Post & 0 Uncoupled Spdl) and 0 SWR detected (0 Pre, 0 Post & 0 Uncoupled SWR)
session5 : starts at 61.7 s & ends at 1356.4 s ( 1294.7 s duration,  0 dropped frames

In [ ]:
Spindles_GlobalResults

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,SpdlStatut,SpdlNumber,SpdlDuration (ms),SWR inside Spdl,CalciumActivityPreference,CalciumActivityBefore,CalciumActivityAfter,AUC_calciumBefore,AUC_calciumAfter,SpikeActivityPreference,SpikeActivityBefore,SpikeActivityAfter,AUC_spikeBefore,AUC_spikeAfter
0,BlackLinesOK,session1,None,3,0,3,[UnCoupled],0,1353.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
1,BlackLinesOK,session1,None,3,0,3,[UnCoupled],1,813.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
2,BlackLinesOK,session1,None,3,0,3,[UnCoupled],2,1032.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
3,BlackLinesOK,session1,None,3,0,3,[UnCoupled],3,773.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
4,BlackLinesOK,session1,None,3,0,3,[UnCoupled],4,1771.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,BlackLinesOK,session4,None,4,3,25,[UnCoupled],0,598.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
379,BlackLinesOK,session4,None,4,3,25,[UnCoupled],1,909.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
380,BlackLinesOK,session4,None,4,3,25,[UnCoupled],2,759.0,False,Before,0.015371,0.012349,0.138298,0.111106,Before,0.015371,0.012349,0.138298,0.111106
381,BlackLinesOK,session4,None,4,3,25,[UnCoupled],3,553.0,False,Before,0.000006,0.000005,0.000056,0.000045,Before,0.000006,0.000005,0.000056,0.000045


Save Spindles_GlobalResults

In [ ]:
mice=os.path.basename(folder_base) 
filenameOut = folder_base / f'Spindles_{Cortex}_ABdetection_GlobalResultsAB_{mice}.xlsx'
writer = pd.ExcelWriter(filenameOut)
Spindles_GlobalResults.to_excel(writer)
writer.close()

Do average results for Spindles

In [ ]:
AVG_dict_All_ActivityCa_spin = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_spin.items()}
Array=list(AVG_dict_All_ActivityCa_spin.values())

AVG_dict_All_ActivityCa_spin_Uncoupled = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_spin_Uncoupled.items()}
ArrayUn=list(AVG_dict_All_ActivityCa_spin_Uncoupled.values())

AVG_dict_All_ActivityCa_spin_Precoupled = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_spin_Precoupled.items()}
ArrayPre=list(AVG_dict_All_ActivityCa_spin_Precoupled.values())

AVG_dict_All_ActivityCa_spin_Postcoupled = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_spin_Postcoupled.items()}
ArrayPost=list(AVG_dict_All_ActivityCa_spin_Postcoupled.values())

filenameOut = folder_base / f'Spindles_{Cortex}_ABdetection_AverageResultsAB_{mice}.xlsx'
excel_writer = pd.ExcelWriter(filenameOut)

Array=pd.DataFrame(Array)
ArrayUn=pd.DataFrame(ArrayUn)
ArrayPre=pd.DataFrame(ArrayPre)
ArrayPost=pd.DataFrame(ArrayPost)

Array.to_excel(excel_writer, sheet_name='All_Spindles', index=True, header=False)
ArrayUn.to_excel(excel_writer, sheet_name='Uncoupled_Spindles', index=True, header=False)
ArrayPre.to_excel(excel_writer, sheet_name='Precoupled_Spindles', index=True, header=False)
ArrayPost.to_excel(excel_writer, sheet_name='Postcoupled_Spindles', index=True, header=False)

excel_writer.close()

In [ ]:
SWR_GlobalResults

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,SWRStatut,SWRNumber,SWRDuration (ms),SWR inside Spdl,CalciumActivityPreference,CalciumActivityBefore,CalciumActivityAfter,AUC_calciumBefore,AUC_calciumAfter,SpikeActivityPreference,SpikeActivityBefore,SpikeActivityAfter,AUC_spikeBefore,AUC_spikeAfter
0,BlackLinesOK,session1,None,3,0,3,[UnCoupled],0,59.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
1,BlackLinesOK,session1,None,3,0,3,[UnCoupled],1,60.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
2,BlackLinesOK,session1,None,3,0,3,[UnCoupled],2,55.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
3,BlackLinesOK,session1,None,3,0,3,[UnCoupled],3,61.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
4,BlackLinesOK,session1,None,3,0,3,[UnCoupled],4,45.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4339,BlackLinesOK,session4,None,4,3,25,[UnCoupled],43,87.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
4340,BlackLinesOK,session4,None,4,3,25,[UnCoupled],44,40.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
4341,BlackLinesOK,session4,None,4,3,25,[UnCoupled],45,39.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0
4342,BlackLinesOK,session4,None,4,3,25,[UnCoupled],46,62.0,False,None,0.0,0.0,0.0,0.0,None,0.0,0.0,0.0,0.0


Save SWR_GlobalResults

In [ ]:
mice=os.path.basename(folder_base) 
filenameOut = folder_base / f'SWR_{Cortex}_ABdetection_GlobalResultsAB_{mice}.xlsx'
writer = pd.ExcelWriter(filenameOut)
SWR_GlobalResults.to_excel(writer)
writer.close()

Do average results for SWR

In [ ]:
AVG_dict_All_ActivityCa_swr = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_swr.items()}
Array=list(AVG_dict_All_ActivityCa_swr.values())

AVG_dict_All_ActivityCa_swr_Uncoupled = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_swr_Uncoupled.items()}
ArrayUn=list(AVG_dict_All_ActivityCa_swr_Uncoupled.values())

AVG_dict_All_ActivityCa_swr_Precoupled = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_swr_Precoupled.items()}
ArrayPre=list(AVG_dict_All_ActivityCa_swr_Precoupled.values())

AVG_dict_All_ActivityCa_swr_Postcoupled = {key: np.mean(matrix,0) for key, matrix in dict_All_ActivityCa_swr_Postcoupled.items()}
ArrayPost=list(AVG_dict_All_ActivityCa_swr_Postcoupled.values())

filenameOut = folder_base / f'SWR_{Cortex}_ABdetection_AverageResultsAB_{mice}.xlsx'
excel_writer = pd.ExcelWriter(filenameOut)

Array=pd.DataFrame(Array)
ArrayUn=pd.DataFrame(ArrayUn)
ArrayPre=pd.DataFrame(ArrayPre)
ArrayPost=pd.DataFrame(ArrayPost)

Array.to_excel(excel_writer, sheet_name='All_SWR', index=True, header=False)
ArrayUn.to_excel(excel_writer, sheet_name='Uncoupled_SWR', index=True, header=False)
ArrayPre.to_excel(excel_writer, sheet_name='Precoupled_SWR', index=True, header=False)
ArrayPost.to_excel(excel_writer, sheet_name='Postcoupled_SWR', index=True, header=False)

excel_writer.close() 